In [1]:
import argparse
import datetime
import json
import tempfile
import os

from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

import sys
import numpy as np
import itertools
import pickle
from random import shuffle
from sklearn.model_selection import StratifiedKFold

/lustre/home/adrianodif/Virtualenvs/keras_hpc/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [71]:
from sklearn.metrics import roc_auc_score

In [27]:
from dataset import *
from model_architectures import *

In [3]:
from keras import backend as K
import tensorflow as tf

In [4]:
remote_data = "/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/"

In [31]:
debug = False

In [32]:
remote_dirs = os.listdir(remote_data)
remote_dirs = [remote_data + d + "/" for d in remote_dirs if os.path.isdir(remote_data + d) and "ten_l" not in d]

shuffle(remote_dirs)

train_dirs = remote_dirs[:int(len(remote_dirs)*0.8)] if not debug else remote_dirs[:1]
test_dirs = remote_dirs[int(len(remote_dirs)*0.8):int(len(remote_dirs)*0.9)] if not debug else remote_dirs[:1]
val_dirs = remote_dirs[int(len(remote_dirs)*0.9):] if not debug else remote_dirs[:1]

In [41]:
train_files = []
val_files = []
test_files = []
print train_dirs
MAX = 1
for ff in train_dirs:
    train_files = train_files + [ff + "/" + f for f in os.listdir(ff) if "Step" in f][:MAX]
for ff in val_dirs:
    val_files = val_files + [ff + "/" + f for f in os.listdir(ff) if "Step" in f][:MAX]
for ff in test_dirs:
    test_files = test_files + [ff + "/" + f for f in os.listdir(ff) if "Step" in f][:MAX]
    
train_files = train_files[:50]
val_files = val_files[:20]
test_files = test_files[:20]

['/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_223545_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_222927_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_223912_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_224210_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_233615_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_222752_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_231628_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_231415_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_223359_runs/', '/lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_233436_runs/', '/lustre/

In [42]:
padshape = 16

target_lab = "label"

pdg_lab = "inTpPdgId"

headLab = ["run","evt","lumi","PU","detSeqIn","detSeqOut","bSX","bSY","bSZ","bSdZ"]

hitCoord = ["X","Y","Z","Phi","R"] #5

hitDet = ["DetSeq","IsBarrel","Layer","Ladder","Side","Disk","Panel","Module","IsFlipped","Ax1","Ax2"] #12

hitClust = ["ClustX","ClustY","ClustSize","ClustSizeX","ClustSizeY","PixelZero",
            "AvgCharge","OverFlowX","OverFlowY","Skew","IsBig","IsBad","IsEdge"] #13

hitPixel = ["Pix" + str(el) for el in range(1, padshape*padshape + 1)]

hitCharge = ["SumADC"]

hitLabs = hitCoord + hitDet + hitClust + hitPixel + hitCharge

inHitLabs = [ "in" + str(i) for i in hitLabs]
outHitLabs = [ "out" + str(i) for i in hitLabs]

inPixels = [ "in" + str(i) for i in hitPixel]
outPixels = [ "out" + str(i) for i in hitPixel]


particle = ["PId","TId","Px","Py","Pz","Pt","MT","ET","MSqr","PdgId",
                "Charge","NTrackerHits","NTrackerLayers","Phi","Eta","Rapidity",
                "VX","VY","VZ","DXY","DZ","BunchCrossing","IsChargeMatched",
                "IsSigSimMatched","SharedFraction","NumAssocRecoTracks"]

hitFeatures = hitCoord + hitDet + hitClust + hitCharge # 5 + 12 + 13 + 1 = 31

inParticle = [ "inTp" + str(i) for i in particle]
outParticle = [ "outTp" + str(i) for i in particle]

inHitFeature  = [ "in" + str(i) for i in hitFeatures]
outHitFeature = [ "out" + str(i) for i in hitFeatures]

particleLabs = ["label","intersect","particles"] + inParticle +  outParticle

differences = ["deltaA", "deltaADC", "deltaS", "deltaR", "deltaPhi", "deltaZ", "ZZero"]

featureLabs = inHitFeature + outHitFeature + differences

dataLab = headLab + inHitLabs + outHitLabs + differences + particleLabs + ["dummyFlag"]

layer_ids = [0, 1, 2, 3, 14, 15, 16, 29, 30, 31]

particle_ids = [-1.,11.,13.,15.,22.,111.,211.,311.,321.,2212.,2112.,3122.,223.]

main_pdgs = [11.,13.,211.,321.,2212.]

layer_ids = [0, 1, 2, 3, 14, 15, 16, 29, 30, 31]

particle_ids = [-1.,11.,13.,15.,22.,111.,211.,311.,321.,2212.,2112.,3122.,223.]

main_pdgs = [11.,13.,211.,321.,2212.]

il1Pix = ['inPix41', 'inPix57', 'inPix72', 'inPix73', 'inPix74', 'inPix88', 'inPix89', 'inPix90', 'inPix104', 'inPix105', 'inPix106', 'inPix119', 'inPix120', 'inPix121', 'inPix122', 'inPix123', 'inPix135', 'inPix136', 'inPix137', 'inPix138', 'inPix139', 'inPix140', 'inPix151', 'inPix152', 'inPix153', 'inPix154', 'inPix155', 'inPix168', 'inPix169', 'inPix170', 'inPix184', 'inPix185', 'inPix186', 'inPix200', 'inPix201', 'inPix202', 'inPix217', 'inPix233']

ol1Pix = ['outPix89', 'outPix104', 'outPix105', 'outPix106', 'outPix119', 'outPix120', 'outPix121', 'outPix122', 'outPix123', 'outPix134', 'outPix135', 'outPix136', 'outPix137', 'outPix138', 'outPix139', 'outPix151', 'outPix152', 'outPix153', 'outPix154', 'outPix155', 'outPix168', 'outPix169', 'outPix170', 'outPix185']

inHitCoords = [ "in" + str(i) for i in hitCoord]
outHitCoords = [ "out" + str(i) for i in hitCoord]
inHitDet = [ "in" + str(i) for i in hitDet]
outHitDet = [ "out" + str(i) for i in hitDet]
inHitClust = [ "in" + str(i) for i in hitClust]
outHitClust = [ "out" + str(i) for i in hitClust]

In [43]:
theInfos = featureLabs

In [44]:
val_data = Dataset(val_files)
test_data = Dataset(test_files)
train_data = Dataset(train_files)

Loading file 1/16 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_224458_runs//_bal_1_1_2_mixedTripletStepHitDoubletsA_dnn_doublets_00.h5
Loading file 2/16 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_224740_runs//_bal_1_1_2_mixedTripletStepHitDoubletsA_dnn_doublets_00.h5
Loading file 3/16 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_223236_runs//_bal_1_1_2_mixedTripletStepHitDoubletsA_dnn_doublets_00.h5
Loading file 4/16 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_221818_runs//_bal_1_1_2_mixedTripletStepHitDoubletsA_dnn_doublets_00.h5
Loading file 5/16 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_233339_runs//_bal_1_1_2_mixedTripletStepHitDoubletsA_dnn_doublets_00.h5
Loading file 6/16 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_232617_runs//_bal_1_1_

Loading file 18/50 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_233634_runs//_bal_1_1_2_mixedTripletStepHitDoubletsA_dnn_doublets_00.h5
Loading file 19/50 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_225715_runs//_bal_1_1_4_lowPtQuadStepHitDoublets_dnn_doublets_02.h5
Loading file 20/50 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_225928_runs//_bal_1_1_2_mixedTripletStepHitDoubletsA_dnn_doublets_00.h5
Loading file 21/50 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_230725_runs//_bal_1_1_4_lowPtQuadStepHitDoublets_dnn_doublets_02.h5
Loading file 22/50 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_231018_runs//_bal_1_1_2_mixedTripletStepHitDoubletsA_dnn_doublets_00.h5
Loading file 23/50 : /lustre/cms/store/user/adiflori/ConvPixels/TTBar_13TeV_PU_35/bal_data/5_35_02May18_232253_runs//_bal_1_1_4_

In [92]:
X_val_hit, X_val_info, y_val = val_data.get_layer_map_data(labs=theInfos)
X_test_hit, X_test_info, y_test = test_data.get_layer_map_data(labs=theInfos)
X_train_hit, X_train_info, y_train = train_data.get_layer_map_data(labs=theInfos)

In [95]:
X_val_hit = X_val_hit.transpose(0,3,1,2)
X_val_hit.shape

(92432, 2, 16, 16)

In [96]:
X_test_hit = X_test_hit.transpose(0,3,1,2)
X_test_hit.shape

(88664, 2, 16, 16)

In [97]:
X_train_hit = X_train_hit.transpose(0,3,1,2)
X_train_hit.shape

(191896, 2, 16, 16)

In [98]:
def pixel_only_model(img_size = 16, n_channels=20):
    hit_shapes = Input(shape=(n_channels,img_size, img_size), name='hit_shape_input')
    
    b_norm = BatchNormalization()(hit_shapes)
    #drop = Dropout(args.dropout)(hit_shapes)
    conv = Conv2D(64, (3, 3), activation='relu', padding='same', data_format="channels_first", name='conv1')(hit_shapes)
    conv = Conv2D(64, (3, 3), activation='relu', padding='same', data_format="channels_first", name='conv2')(conv)
    b_norm = BatchNormalization()(conv)
    pool = MaxPooling2D(pool_size=(2, 2), padding='same', data_format="channels_last", name='pool1')(b_norm)

    conv = Conv2D(128, (3, 3), activation='relu', padding='same', data_format="channels_first", name='conv3')(pool)
    conv = Conv2D(128, (3, 3), activation='relu', padding='same', data_format="channels_first", name='conv4')(conv)
    b_norm = BatchNormalization()(conv)
    pool = MaxPooling2D(pool_size=(2, 2), padding='same', data_format="channels_first", name='pool2')(b_norm)

    flat = Flatten()(pool)

    #drop = Dropout(args.dropout)(flat)
    dense = Dense(128, activation='relu', kernel_constraint=max_norm(1.0), name='dense1')(flat)
    #drop = Dropout(args.dropout)(dense)
    b_norm = BatchNormalization()(dense)
    dense = Dense(64, activation='relu', kernel_constraint=max_norm(1.0), name='dense2')(b_norm)
    #drop = Dropout(args.dropout)(dense)
    pred = Dense(2, activation='softmax', kernel_constraint=max_norm(1.0), name='output')(dense)

    model = Model(inputs=hit_shapes, outputs=pred)
    #my_sgd = optimizers.SGD(lr=args.lr, decay=1e-4, momentum=args.momentum, nesterov=True)
    model.compile(optimizer="adam", loss='categorical_crossentropy', metrics=['accuracy'])
    return model



In [99]:
model = pixel_only_model(n_channels=X_train_hit.shape[1]) 

In [100]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
hit_shape_input (InputLayer) (None, 2, 16, 16)         0         
_________________________________________________________________
conv1 (Conv2D)               (None, 64, 16, 16)        1216      
_________________________________________________________________
conv2 (Conv2D)               (None, 64, 16, 16)        36928     
_________________________________________________________________
batch_normalization_16 (Batc (None, 64, 16, 16)        64        
_________________________________________________________________
pool1 (MaxPooling2D)         (None, 32, 8, 16)         0         
_________________________________________________________________
conv3 (Conv2D)               (None, 128, 8, 16)        36992     
_________________________________________________________________
conv4 (Conv2D)               (None, 128, 8, 16)        147584    
__________

In [101]:
callbacks = [
        EarlyStopping(monitor='val_loss', patience=20),
        #ModelCheckpoint(fname + "_last.h5", save_best_only=True,
                        #save_weights_only=True),
        #TensorBoard(log_dir=log_dir_tf, histogram_freq=0,
                    #write_graph=True, write_images=True)
		#roc_callback(training_data=(train_input_list,y),validation_data=(val_input_list,y_val))
    ]

In [102]:

history = model.fit(X_train_hit, y_train, batch_size=2048, epochs=5, shuffle=True,validation_data=(X_val_hit,y_val), callbacks=callbacks, verbose=1)


Train on 191896 samples, validate on 92432 samples
Epoch 1/5
191896/191896 [==============================] - 36s 187us/step - loss: 0.6431 - acc: 0.6398 - val_loss: 0.7063 - val_acc: 0.5563
Epoch 2/5
191896/191896 [==============================] - 31s 160us/step - loss: 0.6060 - acc: 0.6740 - val_loss: 0.6943 - val_acc: 0.6056
Epoch 3/5
191896/191896 [==============================] - 31s 160us/step - loss: 0.5789 - acc: 0.6933 - val_loss: 0.6553 - val_acc: 0.6427
Epoch 4/5
191896/191896 [==============================] - 31s 160us/step - loss: 0.5524 - acc: 0.7149 - val_loss: 0.6498 - val_acc: 0.6560
Epoch 5/5
191896/191896 [==============================] - 31s 160us/step - loss: 0.5175 - acc: 0.7384 - val_loss: 0.6744 - val_acc: 0.6494


In [103]:
def max_binary_accuracy(y_true, y_pred,n=50):

    thresholds = np.linspace(0.0,1.0,num=n)
    accmax = 0
    for t in thresholds:
        acc = np.mean(((y_pred[:,0] > t).astype(float)==y_true[:,0]).astype(float))
        if acc > accmax:
            tmax = t
            accmax = acc
    return accmax,tmax

In [104]:
loss, acc = model.evaluate(X_test_hit, y_test, batch_size=2048)
test_pred = model.predict(X_test_hit)
test_roc = roc_auc_score(y_test, test_pred)
test_acc,t_test = max_binary_accuracy(y_test,test_pred,n=1000)
print('Test loss / test AUC       = {:.4f} / {:.4f} '.format(loss,test_roc))
print('Test acc /  acc max (@t)   = {:.4f} / {:.4f} ({:.3f})'.format(acc,test_acc,t_test))

88664/88664 [==============================] - 5s 53us/step
Test loss / test AUC       = 0.6610 / 0.7073 
Test acc /  acc max (@t)   = 0.6587 / 0.6590 (0.479)


In [129]:
loss, acc = model.evaluate(X_train_hit, y_train, batch_size=1024)
test_pred = model.predict(X_train_hit)
train_roc = roc_auc_score(y_train, test_pred)
train_acc,t_train = max_binary_accuracy(y_train,test_pred,n=100)
print('Train loss / train AUC       = {:.4f} / {:.4f} '.format(loss,train_roc))
print('Train acc /  acc max (@t)   = {:.4f} / {:.4f} ({:.3f})'.format(acc,train_acc,t_train))

191896/191896 [==============================] - 10s 55us/step
Train loss / train AUC       = 0.5326 / 0.8072 
Train acc /  acc max (@t)   = 0.7270 / 0.7317 (0.444)


# Saving Data

In [109]:
X_train_hit[0,0,:,:]

array([[    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
        12480.,     0.,     0.,     0.,     0.,     0.,   

In [121]:
data_linear      = X_train_hit.reshape(X_train_hit.shape[0],-1)
data_linear_val  = X_val_hit.reshape(X_val_hit.shape[0],-1)
data_linear_test = X_test_hit.reshape(X_test_hit.shape[0],-1)

In [111]:
data_linear.shape

(191896, 512)

In [112]:
data_linear[0][0:256]

array([    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
           0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
       12480.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
      

In [113]:
data_linear[0][0:256].reshape(16,16)

array([[    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
            0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.],
       [    0.,     0.,     0.,     0.,     0.,     0.,     0.,     0.,
        12480.,     0.,     0.,     0.,     0.,     0.,   

In [141]:
data_linear_train_df = pd.DataFrame(data_linear,columns=inPixels+outPixels)
data_linear_train_df["y0"] = y_train[:,0]
data_linear_train_df["y1"] = y_train[:,1]
data_linear_train_df.head()
data_linear_test_df = pd.DataFrame(data_linear_test,columns=inPixels+outPixels)
data_linear_test_df["y0"] = y_test[:,0]
data_linear_test_df["y1"] = y_test[:,1]
data_linear_test_df.head()
data_linear_val_df = pd.DataFrame(data_linear_val,columns=inPixels+outPixels)
data_linear_val_df["y0"] = y_val[:,0]
data_linear_val_df["y1"] = y_val[:,1]
data_linear_val_df.head()

,inPix1,inPix2,inPix3,inPix4,inPix5,inPix6,inPix7,inPix8,inPix9,inPix10,...,outPix249,outPix250,outPix251,outPix252,outPix253,outPix254,outPix255,outPix256,y0,y1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [155]:
data_linear_val_df.to_hdf("pixel_only_data_val.h5","data",append=False)
data_linear_test_df.to_hdf("pixel_only_data_test.h5","data",append=False)
data_linear_train_df.to_hdf("pixel_only_data_train.h5","data",append=False)

In [125]:
def freeze_session(session, keep_var_names=None, output_names=None, clear_devices=True):
    from tensorflow.python.framework.graph_util import convert_variables_to_constants
    graph = session.graph
    with graph.as_default():
        freeze_var_names = list(set(v.op.name for v in tf.global_variables()).difference(keep_var_names or []))
        output_names = output_names or []
        output_names += [v.op.name for v in tf.global_variables()]
        input_graph_def = graph.as_graph_def()
        if clear_devices:
            for node in input_graph_def.node:
                node.device = ""
        frozen_graph = convert_variables_to_constants(session, input_graph_def,
                                                      output_names, freeze_var_names)
        return frozen_graph

# Saving model and test saved

In [126]:
model.save_weights("pixel_only_final.h5", overwrite=True)
frozen_graph = freeze_session(K.get_session(),
                              output_names=[out.op.name for out in model.outputs])
tf.train.write_graph(frozen_graph, "./", "pixel_only_final.pb", as_text=False)

INFO:tensorflow:Froze 337 variables.
Converted 337 variables to const ops.


'./pixel_only_final.pb'

In [127]:
model = pixel_only_model(n_channels=X_train_hit.shape[1]) 

In [128]:
model.load_weights("pixel_only_final.h5")

In [143]:
test_df = pd.read_hdf("pixel_only_data_test.h5")

In [144]:
test_df.head()

,inPix1,inPix2,inPix3,inPix4,inPix5,inPix6,inPix7,inPix8,inPix9,inPix10,...,outPix249,outPix250,outPix251,outPix252,outPix253,outPix254,outPix255,outPix256,y0,y1
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [149]:
X_test_hist = test_df[inPixels+outPixels].values.reshape(-1,2,16,16)
X_test_hist.shape

(88664, 2, 16, 16)

In [150]:
y_test = test_df[["y0","y1"]].values.reshape(-1,2)

In [151]:
loss, acc = model.evaluate(X_test_hit, y_test, batch_size=2048)
test_pred = model.predict(X_test_hit)
test_roc = roc_auc_score(y_test, test_pred)
test_acc,t_test = max_binary_accuracy(y_test,test_pred,n=1000)
print('Test loss / test AUC       = {:.4f} / {:.4f} '.format(loss,test_roc))
print('Test acc /  acc max (@t)   = {:.4f} / {:.4f} ({:.3f})'.format(acc,test_acc,t_test))

88664/88664 [==============================] - 5s 51us/step
Test loss / test AUC       = 0.6610 / 0.7073 
Test acc /  acc max (@t)   = 0.6587 / 0.6590 (0.479)


In [156]:
print "Zipping"
!gzip -1 pixel_only_data*.h5

Zipping


In [157]:
!pwd

/lustre/home/adrianodif/CNNDoublets/jupyters
